In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
# import necessary packages
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [4]:
sc = SparkContext()
# we initiate the StreamingContext with 10 second batch interval. #next we initiate our sqlcontext
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

c:\Users\I539022\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# initiate streaming text from a TCP (socket) source:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)
# lines of tweets with socket_stream window of size 60, or 60 #seconds windows of time
lines = socket_stream.window(60)

In [6]:
# just a tuple to assign names
from collections import namedtuple

fields = ("hashtag", "count")
Tweet = namedtuple('Tweet', fields)
# here we apply different operations on the tweets and save them to #a temporary sql table
(lines.flatMap(lambda text: text.split(" "))  #Splits to a list
 # Checks for    hashtag calls
 .filter(lambda word: word.lower().startswith("#")).map(
     lambda word: (word.lower(), 1))  # Lower cases the word
 .reduceByKey(lambda a, b: a + b)
 # Stores in a Tweet Object
 .map(lambda rec: Tweet(rec[0], rec[1]))
 # Sorts Them in a dataframe
 .foreachRDD(lambda rdd: rdd.toDF().sort(desc("count"))
             # Registers only top 10 hashtags to a table.
             .limit(10).registerTempTable("tweets")))


In [7]:
# start streaming and wait couple of minutes to get enought tweets
ssc.start()

In [8]:
# import libraries to visualize the results
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas

get_ipython().run_line_magic('matplotlib', 'inline')
count = 0
while count < 5:

    time.sleep(5)
    top_10_tags = sqlContext.sql('Select hashtag, count from tweets')
    top_10_df = top_10_tags.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize=(10, 8))
    sns.barplot(x="count", y="hashtag", data=top_10_df)
    plt.show()
    count = count + 1
    print(count)

Py4JJavaError: An error occurred while calling o29.sql.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6

In [ ]:
# stop streaming and wait couple of minutes to get enought tweets
ssc.stop()